In [1]:
import os
os.chdir('../')
%pwd

'/home/fibonacci/projects/plant-disease-prediction'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir:Path
    tensorboard_root_log_dir:Path
    checkpoint_model_filepath:Path

In [3]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml,create_directories

In [5]:
class ConfigurationManager:
    def __init__(self,config_filepath=CONFIG_FILE_PATH,params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])
    def get_prepare_callbacks_config(self)-> PrepareCallbacksConfig:
        config=self.config.prepare_callbacks
        model_ckpt_dir=os.path.dirname(config.checkpoint_model_filepath)
        create_directories([
            Path(model_ckpt_dir),
            Path(config.tensorboard_root_log_dir)
        ])
        prepare_callback_config=PrepareCallbacksConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath)
        )
        return prepare_callback_config

In [6]:
import os
import urllib.request as request
import zipfile
import tensorflow as tf
import time

2024-02-16 12:06:50.022752: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-16 12:06:53.488790: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-16 12:06:53.488970: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-16 12:06:53.872100: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-16 12:06:55.230389: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-16 12:07:04.134991: W tensorflow/compiler/tf2tensorrt/utils/py_utils.

[2024-02-16 12:07:21,488:INFO:utils:NumExpr defaulting to 4 threads.]


In [7]:
class PrepareCallbacks:
    def __init__(self,config:PrepareCallbacksConfig):
        self.config=config
    
    @property
    def _create_tb_callbacks(self):
        timestamp=time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_dir=os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}",      
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_dir)
    
    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=self.config.checkpoint_model_filepath,
            save_weights_only=True,
            save_best_only=True
        )
    def get_tb_ckpt_callbacks(self):
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ]

In [9]:
try:
    config=ConfigurationManager()
    prepare_callbacks_config=config.get_prepare_callbacks_config()
    prepare_callbacks=PrepareCallbacks(config=prepare_callbacks_config)
    callbacks_list=prepare_callbacks.get_tb_ckpt_callbacks()
except Exception as e:
    print(e)


[2024-02-16 12:08:11,722:INFO:common:yaml file:config/config.yaml load successfully]
[2024-02-16 12:08:11,728:INFO:common:yaml file:params.yaml load successfully]
[2024-02-16 12:08:11,730:INFO:common:created directory at:artifacts]


[2024-02-16 12:08:11,850:INFO:common:created directory at:artifacts/prepare_callbacks/checkpoint_dir]
[2024-02-16 12:08:11,855:INFO:common:created directory at:artifacts/prepare_callbacks/tensorboard_log_dir]
